# Working with raster data

*This notebook is based on the "Introduction to Python GIS -course 2018", University of Helsinki (https://automating-gis-processes.github.io/CSC/index.html), CC-BY-SA and GNU GPLv3 license (code)*

In [ ]:
%matplotlib inline

import os

## Reading raster files with Rasterio

[Rasterio](https://mapbox.github.io/rasterio/):

* Pythonic bindings to GDAL and its data model
* Advanced reading and writing of raster files, but also processing tools (masking, reprojection, resampling, ..)
* Integration with numpy and scientific stack

---

[Rasterio](https://mapbox.github.io/rasterio/) is a highly useful module for raster processing which you can use for reading and writing [several different raster formats](http://www.gdal.org/formats_list.html) in Python. Rasterio is based on [GDAL](http://www.gdal.org/) and Python automatically registers all known GDAL drivers for reading supported
formats when importing the module. Most common file formats include for example [TIFF and GeoTIFF](http://www.gdal.org/frmt_gtiff.html),
[ASCII Grid](http://www.gdal.org/frmt_various.html#AAIGrid) and [Erdas Imagine .img](http://www.gdal.org/frmt_hfa.html) -files.

### Download data

**Download the data** package from [here](http://www.helsinki.fi/science/accessibility/opetus/autogis/L5_data.zip). The package contains various TIF-files that will be explored during the tutorials. 

[Landsat 8](http://landsat.gsfc.nasa.gov/landsat-8/landsat-8-bands) bands are stored as separate GeoTIFF -files in the original package. Each band contains information of surface reflectance from different ranges of the electromagnetic spectrum.


Let's start with inspecting one of the files we downloaded:


In [ ]:
!wget "https://raw.githubusercontent.com/Automating-GIS-processes/CSC18/master/data/Helsinki_masked_p188r018_7t20020529_z34__LV-FIN.tif" -P data

In [ ]:
import rasterio

In [ ]:
# Data dir
data_dir = "data"
fp = os.path.join(data_dir, "Helsinki_masked_p188r018_7t20020529_z34__LV-FIN.tif")

In [ ]:
# Open the file:
raster = rasterio.open(fp)

In [ ]:
# Check type of the variable 'raster'
type(raster)

Okey so from here we can see that our `raster` variable is a `rasterio.io.DatasetReader` type which means that we have opened the file for reading.

### Raster file properties

Let's have a closer look at the properties of the file:

In [ ]:
# Driver (data format)
raster.driver

In [ ]:
# Dimensions
print(raster.width)
print(raster.height)

In [ ]:
# Number of bands
raster.count

**Georeferencing**

In [ ]:
# Bounds of the file
raster.bounds

In [ ]:
# Affine transform (mapping pixel locations to coordinates)
raster.transform

In [ ]:
# Projection
raster.crs

All Metadata for the whole raster dataset

In [ ]:
raster.meta

### Get raster bands

Different bands of a satellite images are often stacked together in one raster dataset. In our case, all seven bands of the Landsat 8 scene are included in our GeoTIFF and the `count` is hence 7.

In order to have a closer look at the values stored in the band, we will take advantage of the [GDAL Band API](http://gdal.org/python/osgeo.gdal.Band-class.html).

In [ ]:
# Read the raster band as separate variable
band1 = raster.read(1)

In [ ]:
# Check type of the variable 'band'
print(type(band1))

In [ ]:
# Data type of the values
print(band1.dtype)

Now we have the values of the raster band stored in the variable `band1`.

Quickly visualizing this first band:

In [ ]:
import rasterio.plot

In [ ]:
rasterio.plot.show((raster, 1))

## Intermezzo - Numpy: fundamental array computing package for Python

[NumPy](https://www.numpy.org/) is the fundamental package for scientific computing with Python. It contains among other things:

* a powerful N-dimensional array/vector/matrix object
* sophisticated (broadcasting) functions
* function implementation in C/Fortran assuring good performance if vectorized
* tools for integrating C/C++ and Fortran code
* useful linear algebra, Fourier transform, and random number capabilities

Also known as *array oriented computing*. The recommended convention to import numpy is:

In [ ]:
import numpy as np

**Why is it useful?**

Memory-efficient container that provides fast numerical operations:

In [ ]:
L = range(1000)
%timeit [i**2 for i in L]

In [ ]:
a = np.arange(1000)
%timeit a**2

### Quick primer

In [ ]:
a = np.random.rand(32, 32, 3)

In [ ]:
plt.imshow(a)

**Data attributes**

In [ ]:
a.dtype

In [ ]:
a.shape

In [ ]:
a.ndim

**Indexing and slicing**

In [ ]:
a[:, :, 0].shape

In [ ]:
plt.imshow(a[:, :, 0])

**Element-wise operations and broadcasting**

In [ ]:
arr1 = a[:, :, 0]

In [ ]:
arr1 + 10

In [ ]:
arr1 > 0.5

In [ ]:
a + np.array([0, 1, 2])

**Reductions**

In [ ]:
arr1.mean()

In [ ]:
arr1.min()

Good reference: http://scipy-lectures.org/

## Band statistics

Next, let's have a look at the values that are stored in the band. As the values of the bands are stored as numpy arrays, it is extremely easy to calculate basic statistics by using basic numpy functions.

In [ ]:
# Read all bands
array = raster.read()

In [ ]:
array.shape

In [ ]:
# Calculate statistics for each band
stats = []
for band in array:
    stats.append({
        'min': band.min(),
        'mean': band.mean(),
        'median': np.median(band),
        'max': band.max()})

# Show stats for each channel
stats

## Raster map algebra

Conducting calculations between bands or raster is another common GIS task. Here, we will be calculating `NDVI` (Normalized difference vegetation index) based on the Landsat dataset that we have downloaded from Helsinki region. Conducting calculations with rasterio is fairly straightforward if the extent etc. matches because the values of the rasters are stored as `numpy` arrays (similar to the columns stored in Geo/Pandas, i.e. `Series`).

### Calculating NDVI 

In this tutorial, we will see how to calculate the NDVI (Normalized difference vegetation index) based on two bands: band-4 which is the Red channel and band-5 which is the Near Infrared (NIR).

- Let's read the red and NIR bands from our raster source ([ref](https://etsin.avointiede.fi/storage/f/paituli/latuviitta/Landsat_kanavat.pdf)):

In [ ]:
# Read red channel (channel number 3)
red = raster.read(3)
# Read NIR channel (channel number 4)
nir = raster.read(4)

In [ ]:
# Calculate some stats to check the data
print(red.mean())
print(nir.mean())
print(type(nir))

In [ ]:
# Visualize
rasterio.plot.show(nir, cmap='terrain')

As we can see the values are stored as `numpy.ndarray`. From the map we can see that NIR channel reflects stronly (light green) in areas outside the Helsinki urban areas.

- Let's change the data type from uint8 to float so that we can have floating point numbers stored in our arrays:

In [ ]:
# Convert to floats
red = red.astype('f4')
nir = nir.astype('f4')
nir

Now we can see that the numbers changed to decimal numbers (there is a dot after the zero).

Next we need to tweak the behaviour of numpy a little bit. By default numpy will complain about dividing with zero values. We need to change that behaviour because we have a lot of 0 values in our data.


In [ ]:
np.seterr(divide='ignore', invalid='ignore')

- Now we are ready to calculate the NDVI. This can be done easily with simple map algebra and using the NDVI formula and passing our numpy arrays into it:

In [ ]:
# Calculate NDVI using numpy arrays
ndvi = (nir - red) / (nir + red)

- Let's plot the results so we can see how the index worked out:

In [ ]:
%matplotlib inline
# Plot the NDVI
plt.imshow(ndvi, cmap='terrain_r')
# Add colorbar to show the index
plt.colorbar()

As we can see from the map, now the really low NDVI indices are located in water and urban areas (middle of the map) whereas the areas colored with green have a lot of vegetation according our NDVI index. 

# Read Cloud Optimized Geotiffs

The following materials are based on [this tutorial](https://geohackweek.github.io/raster/04-workingwithrasters/). Read more from that tutorial until this one get's better updated.

- Let's read a Landsat TIF profile from AWS cloud storage:

In [ ]:
# Specify the path for Landsat TIF on AWS
fp = 'http://landsat-pds.s3.amazonaws.com/c1/L8/042/034/LC08_L1TP_042034_20170616_20170629_01_T1/LC08_L1TP_042034_20170616_20170629_01_T1_B4.TIF'

# See the profile
with rasterio.open(fp) as src:
    print(src.profile)

- Let's plot a low resolution overview:

In [ ]:
# Open the COG
with rasterio.open(fp) as src:
    # List of overviews from biggest to smallest
    oviews = src.overviews(1) 
    
    # Retrieve the smallest thumbnail
    oview = oviews[-1] 
    print('Decimation factor= {}'.format(oview))
    # NOTE this is using a 'decimated read' (http://rasterio.readthedocs.io/en/latest/topics/resampling.html)
    thumbnail = src.read(1, out_shape=(1, int(src.height // oview), int(src.width // oview)))

print('array type: ',type(thumbnail))
print(thumbnail)

plt.imshow(thumbnail)
plt.colorbar()
plt.title('Overview - Band 4 {}'.format(thumbnail.shape))
plt.xlabel('Column #')
plt.ylabel('Row #')

- Let's take a subset from high resolution image:

In [ ]:
#https://rasterio.readthedocs.io/en/latest/topics/windowed-rw.html
#rasterio.windows.Window(col_off, row_off, width, height)
window = rasterio.windows.Window(1024, 1024, 1280, 2560)

with rasterio.open(fp) as src:
    subset = src.read(1, window=window)

plt.figure(figsize=(6,8.5))
plt.imshow(subset)
plt.colorbar(shrink=0.5)
plt.title('Band 4 Subset\n{window}')
plt.xlabel('Column #')
plt.ylabel('Row #')

These commands demonstrate the basics how to use COGs to retrieve data from the cloud.